In [96]:
import requests
import pandas as pd
import json
from dateutil import parser

`API_KEY` - Your api key
`ACCOUNT_ID` - Oanda account ID

In [38]:
API_KEY = "0bc5b85d1139ba82c1beecc839e712e7-c4af67cee06820f36ef3d593098af9cb"
ACCOUNT_ID = "101-012-20651332-001"
OANDA_URL = "	https://api-fxpractice.oanda.com/"

Making a new session

In [39]:
session = requests.Session()

Making the headers for Authorization and the type of content we will receive

In [40]:
session.headers.update({
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type" : "application/json"
})

Putting some params based on Onada Api docs - https://developer.oanda.com/rest-live-v20/instrument-ep/

In [41]:
params = dict(
    count = 10,
    granularity = "H1",
    price = "MBA"
)

Fetching the json file and getting the response for all the tradable instruments

In [45]:
url = f"{OANDA_URL}v3/accounts/{ACCOUNT_ID}/instruments"

In [46]:
response = session.get(url, params=None, data=None, headers=None)

In [54]:
data = response.json()

In [58]:
instruments_list = data['instruments']

Getting the information from the provided instruments like name, type ect..

In [61]:
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [62]:
key_i = ['name', 'type', 'displayName', 'pipLocation', 
         'displayPrecision', 'tradeUnitsPrecision','marginRate']

In [66]:
instruments_dict = {}
for i in instruments_list:
    key = i['name']
    instruments_dict[key] = {k: i[k] for k in key_i}

In [64]:
instruments_dict

{'CHINAH_HKD': {'name': 'CHINAH_HKD',
  'type': 'CFD',
  'displayName': 'China H Shares',
  'pipLocation': 0,
  'displayPrecision': 1,
  'tradeUnitsPrecision': 1,
  'marginRate': '0.1'},
 'AU200_AUD': {'name': 'AU200_AUD',
  'type': 'CFD',
  'displayName': 'Australia 200',
  'pipLocation': 0,
  'displayPrecision': 1,
  'tradeUnitsPrecision': 1,
  'marginRate': '0.05'},
 'USD_PLN': {'name': 'USD_PLN',
  'type': 'CURRENCY',
  'displayName': 'USD/PLN',
  'pipLocation': -4,
  'displayPrecision': 5,
  'tradeUnitsPrecision': 0,
  'marginRate': '0.05'},
 'GBP_AUD': {'name': 'GBP_AUD',
  'type': 'CURRENCY',
  'displayName': 'GBP/AUD',
  'pipLocation': -4,
  'displayPrecision': 5,
  'tradeUnitsPrecision': 0,
  'marginRate': '0.05'},
 'SG30_SGD': {'name': 'SG30_SGD',
  'type': 'CFD',
  'displayName': 'Singapore 30',
  'pipLocation': -1,
  'displayPrecision': 2,
  'tradeUnitsPrecision': 1,
  'marginRate': '0.1'},
 'EUR_USD': {'name': 'EUR_USD',
  'type': 'CURRENCY',
  'displayName': 'EUR/USD',
  

In [68]:
with open("../Data/instuments.json", "w") as f:
    f.write(json.dumps(instruments_dict, indent=2))

In [109]:
def fetch_candle(pair_name, count=10, granularity = "H1"):
    '''Functions that takes pair name count and granularity and 
        returns the status code and the candle data for the instrument in json.'''
    
    url = f"{OANDA_URL}v3/instruments/{pair_name}/candles"
    
    params = dict(
    count = count,
    granularity = granularity,
    price = "MBA"
    )
    
    response = session.get(url, params=params, data=None, headers=None)
    data = response.json()
    
    if response.status_code == 200:
        if "candles"not in data:
            data = []
        else:
            data = data["candles"]
            
    return response.status_code, data


def get_candles_df(data):
    
    if len(data) == 0:
        return pd.DataFrame()
    
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']
    final_data = []

    for candle in data:
        if candle['complete'] == False:
            continue
        new_dict = {}

        new_dict['time'] = parser.parse(candle['time'])
        new_dict['volume'] = candle['volume']

        for p in prices:
            for o in ohlc:
                new_dict[f"{p}_{o}"] = float(candle[p][o])

        final_data.append(new_dict)

    df = pd.DataFrame.from_dict(final_data)
    return df

In [112]:
code, data = fetch_candle("EUR_USD", count=10, granularity="H4")
candles_df = get_candles_df(data)

In [113]:
candles_df

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2022-07-06 05:00:00+00:00,32892,1.02472,1.02768,1.02264,1.02444,1.02465,1.02759,1.02256,1.02435,1.02480,1.02776,1.02273,1.02453
1,2022-07-06 09:00:00+00:00,34253,1.02446,1.02446,1.01662,1.01873,1.02437,1.02438,1.01655,1.01866,1.02454,1.02455,1.01670,1.01880
2,2022-07-06 13:00:00+00:00,39226,1.01871,1.02014,1.01615,1.01729,1.01864,1.02007,1.01607,1.01722,1.01878,1.02020,1.01622,1.01736
3,2022-07-06 17:00:00+00:00,24121,1.01728,1.01988,1.01720,1.01851,1.01720,1.01979,1.01711,1.01840,1.01736,1.01997,1.01726,1.01862
4,2022-07-06 21:00:00+00:00,9576,1.01824,1.01890,1.01761,1.01871,1.01798,1.01883,1.01752,1.01863,1.01850,1.01898,1.01770,1.01879
5,2022-07-07 01:00:00+00:00,16016,1.01872,1.02100,1.01805,1.02018,1.01864,1.02092,1.01797,1.02010,1.01881,1.02108,1.01813,1.02027
6,2022-07-07 05:00:00+00:00,27670,1.02018,1.02212,1.01822,1.01978,1.02011,1.02204,1.01813,1.01970,1.02026,1.02221,1.01829,1.01987
7,2022-07-07 09:00:00+00:00,32126,1.01976,1.02176,1.01696,1.01702,1.01968,1.02169,1.01689,1.01695,1.01984,1.02185,1.01703,1.01710
8,2022-07-07 13:00:00+00:00,27796,1.01702,1.01935,1.01510,1.01552,1.01696,1.01928,1.01503,1.01546,1.01709,1.01942,1.01517,1.01558
